This notebook contains Helen's code merging the combined FEC / DC Inbox data set.

In [100]:
# import libraries
import pandas as pd
import numpy as np
import regex as re

In [101]:
# Read in the raw DCInbox table

dcinbox_raw_df = pd.read_csv(
    "../data/dcinbox/dcinbox_export_116.csv",
    # sep = ",",            # columns are separated by commas
    encoding = "latin1",
    quotechar = '"',   # Make sure actual quotation marks in newsletters are handled right
    engine = 'python'    # This might be able to handle very long messages better
)

# Remove all unnamed columns, because they're messing up the read of the file
unnamed = dcinbox_raw_df.columns.str.contains('^Unnamed')
dcinbox_clean_columns_df = dcinbox_raw_df.loc[:, ~unnamed].copy()

print(len(dcinbox_clean_columns_df))
dcinbox_clean_columns_df.head()

30872


,Subject,Body,Unix Timestamp,BioGuide ID,Congress,First Name,Last Name,Date of Birth,Gender,State,District,Party,Chamber,Nickname,ID
0,The Leader's Daily Schedule - 1/3/21,Kevin McCarthy - Republican Leader Leader's Da...,1.60963E+12,M001165,116,Kevin,McCarthy,1/26/65,M,CA,23,Republican,House,NaN,176119
1,Join me (virtually) for the swearing-in of the...,Join me (virtually) for the swearing-in of th...,1.60961E+12,H001066,116,Steven,Horsford,4/29/73,M,NV,4,Democrat,House,NaN,176120
2,"2020, A Year in Review","2020, A Year in Review Looking Back on 2020 2...",1.60959E+12,G000576,116,Glenn,Grothman,7/3/55,M,WI,6,Republican,House,NaN,176121
3,RESPONSE REQUESTED: Stay Updated on the 117th ...,RESPONSE REQUESTED: Stay Updated on the 117th...,1.60959E+12,M001190,116,Markwayne,Mullin,7/26/77,M,OK,2,Republican,House,NaN,176122
4,"Thank You TX-11, and Godspeed","Thank You TX-11, and Godspeed January 2, 2021...",1.60958E+12,C001062,116,K.,Conaway,6/11/48,M,TX,11,Republican,House,NaN,176123


In [102]:
# Field cleanup for DC Inbox table

# dcinbox_df = dcinbox_clean_columns_df
dcinbox_df = dcinbox_clean_columns_df.copy()

# Convert unix timestamp to a datetime column
dcinbox_df["Date"] = pd.to_datetime(
    pd.to_numeric(dcinbox_df["Unix Timestamp"], errors="coerce"),
    unit="ms"
).dt.date

# Convert district to an integer and fill senators with 0
dcinbox_df["District"] = (
    pd.to_numeric(dcinbox_df["District"], errors = "coerce")
    .fillna(0)
    .astype(int)
)

# create a column that is the uppercased, concatenated first and last name of the politician
# (for matching later to Open Secrets data)
dcinbox_df["Full Name"] = dcinbox_df["First Name"].str.upper() + " " + dcinbox_df["Last Name"].str.upper()

# Rename party to party full
dcinbox_df.rename(columns={"Party": "Party Full"}, inplace=True)
# Add a new party column that just is one character mapped to D, R, or O (Other)
dcinbox_df["Party"] = (
    dcinbox_df["Party Full"]
    .fillna("")
    .astype(str)
    .str[0]
    .str.upper()
    .map({"D": "D", "R": "R"})
    .fillna("O")
)

# Make sure state and party are uppercased
dcinbox_df["State"] = dcinbox_df["State"].fillna("").str.strip().str.upper()
dcinbox_df["Party"] = dcinbox_df["Party"].str.upper()

print(len(dcinbox_df))
dcinbox_df.head()

30872


,Subject,Body,Unix Timestamp,BioGuide ID,Congress,First Name,Last Name,Date of Birth,Gender,State,District,Party Full,Chamber,Nickname,ID,Date,Full Name,Party
0,The Leader's Daily Schedule - 1/3/21,Kevin McCarthy - Republican Leader Leader's Da...,1.60963E+12,M001165,116,Kevin,McCarthy,1/26/65,M,CA,23,Republican,House,NaN,176119,2021-01-02,KEVIN MCCARTHY,R
1,Join me (virtually) for the swearing-in of the...,Join me (virtually) for the swearing-in of th...,1.60961E+12,H001066,116,Steven,Horsford,4/29/73,M,NV,4,Democrat,House,NaN,176120,2021-01-02,STEVEN HORSFORD,D
2,"2020, A Year in Review","2020, A Year in Review Looking Back on 2020 2...",1.60959E+12,G000576,116,Glenn,Grothman,7/3/55,M,WI,6,Republican,House,NaN,176121,2021-01-02,GLENN GROTHMAN,R
3,RESPONSE REQUESTED: Stay Updated on the 117th ...,RESPONSE REQUESTED: Stay Updated on the 117th...,1.60959E+12,M001190,116,Markwayne,Mullin,7/26/77,M,OK,2,Republican,House,NaN,176122,2021-01-02,MARKWAYNE MULLIN,R
4,"Thank You TX-11, and Godspeed","Thank You TX-11, and Godspeed January 2, 2021...",1.60958E+12,C001062,116,K.,Conaway,6/11/48,M,TX,11,Republican,House,NaN,176123,2021-01-02,K. CONAWAY,R


In [103]:
# Read in the matched politicians table

column_names = ["target_id", "dcinbox_name", "fec_cid", "fec_name",
                "fec_party", "fec_state", "fec_district",
                "fec_cycle", "append_attribute2", "append_attribute3",
                "similarity_score", "matched_scorer"]

matched_pols_df = pd.read_csv(
    "../data/matched/matched_politicians_2020_test.csv",
    encoding = "latin1",
    quotechar = '"',   # Make sure actual quotation marks in newsletters are handled right
    names = column_names,
    header = 0
)

print(len(matched_pols_df))
matched_pols_df.head()

402


,target_id,dcinbox_name,fec_cid,fec_name,fec_party,fec_state,fec_district,fec_cycle,append_attribute2,append_attribute3,similarity_score,matched_scorer
0,NaN,KEVIN MCCARTHY,N00028152,KEVIN MCCARTHY,R,CA,23,2020,NaN,NaN,100.0,WRatio
1,NaN,STEVEN HORSFORD,N00033638,STEVEN HORSFORD,D,NV,4,2020,NaN,NaN,100.0,WRatio
2,NaN,GLENN GROTHMAN,N00036409,GLENN S GROTHMAN,R,WI,6,2020,NaN,NaN,100.0,token_set_ratio
3,NaN,MARKWAYNE MULLIN,N00033410,MARKWAYNE MULLIN,R,OK,2,2020,NaN,NaN,100.0,WRatio
4,NaN,K. CONAWAY,N00026041,MIKE CONAWAY,R,TX,11,2020,NaN,NaN,90.0,partial_ratio


In [104]:
# Read in the summarized FEC candidate contributions file

cand_contribs_raw_df = pd.read_csv(
    "../data/fec/candidate_contributions_2020_test.csv",
    encoding = "latin1"
)

# Drop the columns we don't need
cand_contribs_raw_df = cand_contribs_raw_df.drop(columns=['DistIDRunFor', 'DistIDCurr', "CurrCand", "CycleCand", 
                                                  "RecipCode", "CRPICO", "FECCandID"])

print(len(cand_contribs_raw_df))
cand_contribs_raw_df.head()

122459


,Cycle,CID,FirstLastP,Party Original,NoPacs,State,District,Party,contrib_date,contrib_count,contrib_amount
0,2020,N00030768,MARTHA ROBY,R,,AL,2,R,2019-01-10,1,2500.0
1,2020,N00030768,MARTHA ROBY,R,,AL,2,R,2019-02-25,1,500.0
2,2020,N00030768,MARTHA ROBY,R,,AL,2,R,2019-02-28,2,750.0
3,2020,N00030768,MARTHA ROBY,R,,AL,2,R,2019-03-14,2,1000.0
4,2020,N00030768,MARTHA ROBY,R,,AL,2,R,2019-03-28,1,500.0


In [105]:
# Add averages for year and month to data frame (for normalizing later)

# Rename columns so they won't clash with the DC Inbox data file
cand_contribs_raw_df = cand_contribs_raw_df.rename(columns={
    "Cycle": "fec_cycle",
    "CID": "fec_cid",
    "FirstLastP": "fec_name",
    "Party Original": "fec_party_original",
    "NoPacs": "no_pacs",
    "State": "fec_state",
    "District": "fec_district",
    "Party": "fec_party"    
})

# Set correct types on some columns
cand_contribs_raw_df["contrib_count"] = cand_contribs_raw_df["contrib_count"].fillna(0)
cand_contribs_raw_df["contrib_amount"] = cand_contribs_raw_df["contrib_amount"].fillna(0).astype(int)

# Set contrib_date to datetime
cand_contribs_raw_df["contrib_date"] = pd.to_datetime(cand_contribs_raw_df["contrib_date"])

# Define the full date range
start_date = pd.Timestamp("2019-01-03")
end_date = pd.Timestamp("2021-01-02")
num_days = (end_date - start_date).days + 1  # inclusive

# Filter to target period
mask = (cand_contribs_raw_df["contrib_date"] >= start_date) & (cand_contribs_raw_df["contrib_date"] <= end_date)
cand_contribs_df = cand_contribs_raw_df.loc[mask].copy()

# Groupby for full-period averages
total_stats = (
    cand_contribs_df.groupby(["fec_name", "fec_cycle", "fec_state", "fec_district", "fec_party"], dropna=False)
    .agg(
        total_contrib_count=("contrib_count", "sum"),
        total_contrib_amount=("contrib_amount", "sum")
    )
    .reset_index()
)

# Calculate full-period daily averages
total_stats["avg_daily_contrib_count"] = total_stats["total_contrib_count"] / num_days
total_stats["avg_daily_contrib_amount"] = total_stats["total_contrib_amount"] / num_days

# Merge in full-period averages
cand_contribs_df = cand_contribs_df.merge(
    total_stats[["fec_name", "fec_cycle", "fec_state", "fec_district", "fec_party", 
                 "avg_daily_contrib_count", "avg_daily_contrib_amount"]],
    on=["fec_name", "fec_cycle", "fec_state", "fec_district", "fec_party"],
    how="left"
)

# Monthly averages
cand_contribs_df['month_start'] = cand_contribs_df['contrib_date'].values.astype('datetime64[M]')

monthly_stats = (
    cand_contribs_df.groupby(["fec_name", "fec_cycle", "fec_state", "fec_district", "fec_party", "month_start"], dropna=False)
    .agg(
        monthly_contrib_count=('contrib_count', 'sum'),
        monthly_contrib_amount=('contrib_amount', 'sum')
    )
    .reset_index()
)

monthly_stats['days_in_month'] = monthly_stats['month_start'].dt.days_in_month
monthly_stats['avg_month_daily_contrib_count'] = monthly_stats['monthly_contrib_count'] / monthly_stats['days_in_month']
monthly_stats['avg_month_daily_contrib_amount'] = monthly_stats['monthly_contrib_amount'] / monthly_stats['days_in_month']

# Merge in monthly averages
cand_contribs_df = cand_contribs_df.merge(
    monthly_stats[['fec_name', 'fec_cycle', 'fec_state', 'fec_district', 'fec_party', 'month_start',
                   'avg_month_daily_contrib_count', 'avg_month_daily_contrib_amount']],
    on=['fec_name', 'fec_cycle', 'fec_state', 'fec_district', 'fec_party', 'month_start'],
    how='left'
)

# Calculate to-date daily averages
cand_contribs_df = cand_contribs_df.sort_values(
    ['fec_name', 'fec_cycle', 'fec_state', 'fec_district', 'fec_party', 'contrib_date']
).copy()

cand_contribs_df['days_since_start'] = (cand_contribs_df['contrib_date'] - start_date).dt.days + 1

cand_contribs_df['cumsum_contrib_count'] = cand_contribs_df.groupby(
    ['fec_name', 'fec_cycle', 'fec_state', 'fec_district', 'fec_party']
)['contrib_count'].cumsum()

cand_contribs_df['cumsum_contrib_amount'] = cand_contribs_df.groupby(
    ['fec_name', 'fec_cycle', 'fec_state', 'fec_district', 'fec_party']
)['contrib_amount'].cumsum()

cand_contribs_df['avg_daily_contrib_count_td'] = cand_contribs_df['cumsum_contrib_count'] / cand_contribs_df['days_since_start']
cand_contribs_df['avg_daily_contrib_amount_td'] = cand_contribs_df['cumsum_contrib_amount'] / cand_contribs_df['days_since_start']

# Drop temporary columns
cand_contribs_df = cand_contribs_df.drop(columns=['cumsum_contrib_count', 'cumsum_contrib_amount', 'days_since_start'])

cand_contribs_df.head()

,fec_cycle,fec_cid,fec_name,fec_party_original,no_pacs,fec_state,fec_district,fec_party,contrib_date,contrib_count,contrib_amount,avg_daily_contrib_count,avg_daily_contrib_amount,month_start,avg_month_daily_contrib_count,avg_month_daily_contrib_amount,avg_daily_contrib_count_td,avg_daily_contrib_amount_td
82710,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-04,2,500,8.239398,5672.162791,2019-01-01,1.354839,1109.677419,1.000000,250.000000
82711,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-08,2,500,8.239398,5672.162791,2019-01-01,1.354839,1109.677419,0.666667,166.666667
82712,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-16,1,100,8.239398,5672.162791,2019-01-01,1.354839,1109.677419,0.357143,78.571429
82713,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-18,2,1000,8.239398,5672.162791,2019-01-01,1.354839,1109.677419,0.437500,131.250000
82714,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-21,2,200,8.239398,5672.162791,2019-01-01,1.354839,1109.677419,0.473684,121.052632


In [106]:
# Add text-parsing columns to dc inbox df, including word count, fundraising term count, and
# ratio of fundraising terms found to total words

# --- Load and clean term list ---
terms_df = pd.read_csv("../data/dcinbox/fundraising_terms.csv")
term_roots = terms_df['term'].dropna().unique().tolist()

# Build a single regex pattern that matches any of the roots, case-insensitive
# Use word boundaries to avoid partial matches inside other words
term_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(t) for t in term_roots) + r')\w*\b', flags=re.IGNORECASE)

# --- Load and process DC Inbox data ---
dcinbox_df['Date'] = pd.to_datetime(dcinbox_df['Date'])

# Create word count column
dcinbox_df['body_word_count'] = dcinbox_df['Body'].str.split().str.len()

# Count fundraising term appearances in Body
def count_terms(text):
    if pd.isna(text):
        return 0
    return len(term_pattern.findall(text))

dcinbox_df['fundraising_terms'] = dcinbox_df['Body'].apply(count_terms)

print("Sample fundraising term counts:")
print(dcinbox_df[['Full Name', 'Date', 'fundraising_terms']].head())

# Compute per-newsletter fundraising ratio
dcinbox_df['fundraising_ratio'] = dcinbox_df.apply(
    lambda x: x['fundraising_terms'] / x['body_word_count'] if x['body_word_count'] > 0 else 0, axis=1
)

# --- Summarize by politician and day ---
dcinbox_summary_df = dcinbox_df.groupby(['Full Name', 'Date'], as_index=False).agg(
    newsletter_count=('Subject', 'count'),
    body_word_count_sum=('body_word_count', 'sum'),
    fundraising_terms_sum=('fundraising_terms', 'sum'),
    fundraising_ratio_mean=('fundraising_ratio', 'mean')  # average ratio per newsletter
)

# Compute total newsletters per politician
dcinbox_summary_df["total_newsletters"] = dcinbox_summary_df.groupby("Full Name")["newsletter_count"].transform("sum")

# Merge back other columns using the first occurrence per day for all other columns
other_cols = [
    col for col in dcinbox_df.columns 
    if col not in ['Subject', 'Body', 'body_word_count', 'Date', 'Full Name', 
                   'Party Truncated', 'fundraising_terms', 'fundraising_ratio']
]
dcinbox_summary_df = dcinbox_summary_df.merge(
    dcinbox_df.groupby(['Full Name', 'Date'])[other_cols].first().reset_index(),
    on=['Full Name', 'Date'],
    how='left'
)

# Keep only House members
dcinbox_summary_df = dcinbox_summary_df[dcinbox_summary_df["Chamber"] == "House"]

# Drop unneeded columns
dcinbox_summary_df = dcinbox_summary_df.drop(
    columns=['Nickname', 'Date of Birth', "First Name", "Last Name", "BioGuide ID", "Unix Timestamp"],
    errors='ignore'
)

print("Processed records:", len(dcinbox_summary_df))
dcinbox_summary_df.head()

Sample fundraising term counts:
          Full Name       Date  fundraising_terms
0    KEVIN MCCARTHY 2021-01-02                  0
1   STEVEN HORSFORD 2021-01-02                  0
2    GLENN GROTHMAN 2021-01-02                  9
3  MARKWAYNE MULLIN 2021-01-02                  0
4        K. CONAWAY 2021-01-02                  0
Processed records: 25785


,Full Name,Date,newsletter_count,body_word_count_sum,fundraising_terms_sum,fundraising_ratio_mean,total_newsletters,Congress,Gender,State,District,Party Full,Chamber,ID,Party
0,A. FERGUSON,2019-01-13,1,357,0,0.000000,44,116,M,GA,3,Republican,House,160063,R
1,A. FERGUSON,2019-01-27,1,528,0,0.000000,44,116,M,GA,3,Republican,House,159679,R
2,A. FERGUSON,2019-02-10,1,457,0,0.000000,44,116,M,GA,3,Republican,House,159236,R
3,A. FERGUSON,2019-03-03,1,430,1,0.002326,44,116,M,GA,3,Republican,House,158641,R
4,A. FERGUSON,2019-03-17,1,271,1,0.003690,44,116,M,GA,3,Republican,House,158135,R


In [107]:
# Joining QC

# Dcinbox file with counts of newsletters by day
print(len(dcinbox_summary_df))

# file with politicians matched between dcinbox and FEC data
print(len(matched_pols_df))

# Summary FEC file of candidates and contributions by day
print(len(cand_contribs_df))


25785
402
122458


In [108]:
# Join summarized DC Inbox and matched politicians data into a new data frame (left join)

# dc_inbox_sum_df = summary_test_df.merge(
dc_inbox_sum_df = dcinbox_summary_df.merge(
    matched_pols_df,
    left_on = ["Full Name"], # , "State", "District", "Party"],
    right_on = ["dcinbox_name"], # "fec_state", "fec_district", "fec_party"],
    how = 'left'
)

# Drop the columns we don't need
dc_inbox_sum_df = dc_inbox_sum_df.drop(columns=['fec_state', 'fec_district', 'fec_party', "target_id", "append_attribute2", "append_attribute3",
                                                "matched_scorer", "Party Full", "dcinbox_name", "ID", "similarity_score"])

# Field cleanup & type correction
# dc_inbox_sum_df["Date"] == "2020-02-01"
# print(summ_df['fec_cycle'] = dc_inbox_sum_df['fec_cycle'].fillna(0).astype(int)
# dc_inbox_sum_df['similarity_score'] = dc_inbox_sum_df['similarity_score'].fillna(0).astype(int)
dc_inbox_sum_df['Date'] = pd.to_datetime(dc_inbox_sum_df['Date'])

# Rename columns so they won't clash with the FEC data file
dc_inbox_sum_df = dc_inbox_sum_df.rename(columns={
    "Full Name": "dc_name",
    "Date": "dc_date",
    "Congress": "dc_congress",
    "Gender": "dc_gender",
    "State": "dc_state",
    "District": "dc_district",
    "Chamber": "dc_chamber",
    "Party": "dc_party"    
})

print(len(dc_inbox_sum_df))
dc_inbox_sum_df.head()
# dc_inbox_sum_df.info()

25785


,dc_name,dc_date,newsletter_count,body_word_count_sum,fundraising_terms_sum,fundraising_ratio_mean,total_newsletters,dc_congress,dc_gender,dc_state,dc_district,dc_chamber,dc_party,fec_cid,fec_name,fec_cycle
0,A. FERGUSON,2019-01-13,1,357,0,0.000000,44,116,M,GA,3,House,R,N00039090,DREW FERGUSON,2020.0
1,A. FERGUSON,2019-01-27,1,528,0,0.000000,44,116,M,GA,3,House,R,N00039090,DREW FERGUSON,2020.0
2,A. FERGUSON,2019-02-10,1,457,0,0.000000,44,116,M,GA,3,House,R,N00039090,DREW FERGUSON,2020.0
3,A. FERGUSON,2019-03-03,1,430,1,0.002326,44,116,M,GA,3,House,R,N00039090,DREW FERGUSON,2020.0
4,A. FERGUSON,2019-03-17,1,271,1,0.003690,44,116,M,GA,3,House,R,N00039090,DREW FERGUSON,2020.0


In [109]:
# Join summarized FEC and DC Inbox into a new data frame (left join) by day & politician,
# For use in computing rolling window contribution amount & count columns

print(len(dc_inbox_sum_df))
print(len(cand_contribs_df))

# Date field type corrections
dc_inbox_sum_df['dc_date'] = pd.to_datetime(dc_inbox_sum_df['dc_date'])
cand_contribs_df['contrib_date'] = pd.to_datetime(cand_contribs_df['contrib_date'])

window_col_join_df = cand_contribs_df.merge(
# joined_all_df = test_dcinbox_df.merge(
    # dc_inbox_sum_df,
    dc_inbox_sum_df.drop(columns=["fec_cycle", "fec_name", "dc_party", "dc_chamber", "dc_district",
                                  "dc_name", "newsletter_count", "body_word_count_sum",
                                  "dc_state", "dc_gender", "dc_congress"]),
    left_on = ['fec_cid', "contrib_date"],
    right_on = ['fec_cid', "dc_date"],
    how = 'left'
)

# Drop the columns we don't need
# window_col_join_df = window_col_join_df.drop(columns=["dc_date"])

# Fill missing counts and amounts with 0 for candidates with no contributions (per day)
# dc_inbox_sum_df['contrib_count'] = dc_inbox_sum_df['body_word_count_sum'].fillna(0).astype(int)
# window_col_join_df['dc_date'] = window_col_join_df['dc_date'].fillna(0)

print(len(window_col_join_df))
window_col_join_df.head()
# dc_inbox_sum_df.info()

25785
122458
122458


,fec_cycle,fec_cid,fec_name,fec_party_original,no_pacs,fec_state,fec_district,fec_party,contrib_date,contrib_count,...,avg_daily_contrib_amount,month_start,avg_month_daily_contrib_count,avg_month_daily_contrib_amount,avg_daily_contrib_count_td,avg_daily_contrib_amount_td,dc_date,fundraising_terms_sum,fundraising_ratio_mean,total_newsletters
0,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-04,2,...,5672.162791,2019-01-01,1.354839,1109.677419,1.000000,250.000000,NaT,NaN,NaN,NaN
1,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-08,2,...,5672.162791,2019-01-01,1.354839,1109.677419,0.666667,166.666667,NaT,NaN,NaN,NaN
2,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-16,1,...,5672.162791,2019-01-01,1.354839,1109.677419,0.357143,78.571429,NaT,NaN,NaN,NaN
3,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-18,2,...,5672.162791,2019-01-01,1.354839,1109.677419,0.437500,131.250000,NaT,NaN,NaN,NaN
4,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-21,2,...,5672.162791,2019-01-01,1.354839,1109.677419,0.473684,121.052632,NaT,NaN,NaN,NaN


In [110]:
# Add a rolling window of three days after the initial date to the joined FEC/DC Inbox data frame

# Set window days
input_window_days = 2

# Use a small test set at first
# window_col_df = window_col_join_df[window_col_join_df["fec_name"].str.contains("PELOSI")].copy()
# window_col_df = window_col_df[window_col_df["contrib_date"] >= "2019-11-19"].copy()

window_col_df = window_col_join_df.copy()

# Make sure date columns are datetime
window_col_df["contrib_date"] = pd.to_datetime(window_col_df["contrib_date"], errors = "coerce")
window_col_df["dc_date"] = pd.to_datetime(window_col_df["dc_date"], errors = "coerce")

# Fill NaNs in contrib columns to avoid issues during summing
window_col_df["contrib_count"] = window_col_df["contrib_count"].fillna(0).astype(int)
window_col_df["contrib_amount"] = window_col_df["contrib_amount"].fillna(0).astype(int)

# Define the group columns (unique identifiers for rolling sums)
group_cols = ["fec_name", "fec_cycle", "fec_state", "fec_district", "fec_party"]
window_col_df[group_cols] = window_col_df[group_cols].fillna("NA")

# Sort by group and date
window_col_df = window_col_df.sort_values(by = group_cols + ["contrib_date"])

# Define a general rolling window function
def add_forward_rolling_sum(df, value_cols, window_days = 2):
    # value_cols should be a list, e.g. ["contrib_count", "contrib_amount"]
    days_col = "window_days"
    df[days_col] = 0  # initialize shared days column

    for value_col in value_cols:
        new_col = f"window_{value_col}"
        df[new_col] = 0

    for name, g in df.groupby(group_cols, sort=False):
        g = g.sort_values("contrib_date")
        contrib_dates = g["contrib_date"].reset_index(drop=True)
        dates = g["dc_date"].reset_index(drop=True)
        results = {val: np.zeros(len(g), dtype=int) for val in value_cols}
        actual_days = np.zeros(len(g), dtype=int)

        for i, current_date in enumerate(dates):

            if pd.isna(current_date):
                actual_days[i] = 0
                for value_col in value_cols:
                    results[value_col][i] = 0
                continue     

            # Default window end
            window_end = current_date + pd.Timedelta(days=window_days)

            # Include all dates within window initially
            mask = (contrib_dates >= current_date) & (contrib_dates <= window_end)

            # Stop before first non-NaT dc_date if found
            next_reset_idx = np.where((dates > current_date) & pd.notna(dates))[0]
            if len(next_reset_idx) > 0:
                first_reset_date = dates[next_reset_idx[0]]
                mask = mask & (contrib_dates < first_reset_date)
                window_end = min(window_end, first_reset_date - pd.Timedelta(days=1))

            # Store actual number of days used
            actual_days[i] = (window_end - current_date).days + 1

            # Compute rolling sums for all specified value columns
            for value_col in value_cols:
                values = g[value_col].fillna(0).astype(int).reset_index(drop=True)
                results[value_col][i] = int(values[mask].sum())

        # Write results back to df
        for value_col in value_cols:
            new_col = f"window_{value_col}"
            df.loc[g.index, new_col] = results[value_col]
        df.loc[g.index, days_col] = actual_days

    return df


# Compute rolling sums for count and amount
window_col_df = add_forward_rolling_sum(
    window_col_df,
    value_cols=["contrib_count", "contrib_amount"],
    window_days = input_window_days
)

# Check results
# joined_all_window_df.head()[["dc_date", "dc_name", "contrib_count", "window_contrib_count", 
#                              "contrib_amount", "window_contrib_amount"]]
window_col_df.head()


,fec_cycle,fec_cid,fec_name,fec_party_original,no_pacs,fec_state,fec_district,fec_party,contrib_date,contrib_count,...,avg_month_daily_contrib_amount,avg_daily_contrib_count_td,avg_daily_contrib_amount_td,dc_date,fundraising_terms_sum,fundraising_ratio_mean,total_newsletters,window_days,window_contrib_count,window_contrib_amount
0,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-04,2,...,1109.677419,1.000000,250.000000,NaT,NaN,NaN,NaN,0,0,0
1,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-08,2,...,1109.677419,0.666667,166.666667,NaT,NaN,NaN,NaN,0,0,0
2,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-16,1,...,1109.677419,0.357143,78.571429,NaT,NaN,NaN,NaN,0,0,0
3,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-18,2,...,1109.677419,0.437500,131.250000,NaT,NaN,NaN,NaN,0,0,0
4,2020,N00040888,ABBY FINKENAUER,D,,IA,1,D,2019-01-21,2,...,1109.677419,0.473684,121.052632,NaT,NaN,NaN,NaN,0,0,0


In [111]:
# QC for joining

test_date = "2020-02-07"
test_name = "SCHIFF"

dc_inbox_sum_df['dc_date'] = pd.to_datetime(dc_inbox_sum_df['dc_date'])
test_dcinbox_df = dc_inbox_sum_df[dc_inbox_sum_df["dc_date"] == test_date]
test_dcinbox_df = test_dcinbox_df[test_dcinbox_df["dc_name"].str.contains(test_name)]

cand_contribs_df['contrib_date'] = pd.to_datetime(cand_contribs_df['contrib_date'])
test_fec_df = cand_contribs_df[cand_contribs_df["contrib_date"] == test_date]
test_fec_df = test_fec_df[test_fec_df["fec_name"].str.contains(test_name)]

print(len(test_dcinbox_df))
print(len(test_fec_df))
# test_dcinbox_df.head()
test_fec_df.head()

1
1


,fec_cycle,fec_cid,fec_name,fec_party_original,no_pacs,fec_state,fec_district,fec_party,contrib_date,contrib_count,contrib_amount,avg_daily_contrib_count,avg_daily_contrib_amount,month_start,avg_month_daily_contrib_count,avg_month_daily_contrib_amount,avg_daily_contrib_count_td,avg_daily_contrib_amount_td
2475,2020,N00009585,ADAM SCHIFF,D,,CA,28,D,2020-02-07,267,42987,71.294118,10319.623803,2020-02-01,160.275862,25381.103448,55.541147,11958.730673


In [113]:
# Join summarized DC Inbox and window-column FEC into a new data frame (left join) by day & politician

print("window_col_df: ", len(window_col_df))
print("dc_inbox_sum_df: ", len(dc_inbox_sum_df))

# Use a small test set at first
# dc_inbox_sum_df = dc_inbox_sum_df[dc_inbox_sum_df["dc_name"].str.contains("FINKENAUER")].copy()
# dc_inbox_sum_df = dc_inbox_sum_df[dc_inbox_sum_df["dc_date"] >= "2020-01-30"].copy()

# Merge by politician and date
joined_all_df = dc_inbox_sum_df.merge(
    window_col_df.drop(columns  =[
        'fec_cycle', 'fec_name', 'fec_party', 'fec_state', 'fec_district', 'dc_date', 'fundraising_terms_sum', 'total_newsletters', 'fundraising_ratio_mean'
    ]),
    left_on = ['fec_cid', 'dc_date'],
    right_on = ['fec_cid', 'contrib_date'],
    how = 'left'
)
print("joined_all_df: ", len(joined_all_df))

# Drop the columns we don't need
# joined_all_df = joined_all_df.drop(columns=['State', 'District', 'Party', "append_attribute2", "append_attribute3"])

# Add score columns for three pairs of count and amount columns
# to get a score of how much better or worse a given day did than average
# Pair 1: normalizing by whole-period average daily performance
joined_all_df["avg_daily_fp_count_score"] = (
    (joined_all_df["window_contrib_count"] / joined_all_df["window_days"]) /
    joined_all_df["avg_daily_contrib_count"]
)
joined_all_df["avg_daily_fp_amount_score"] = (
    (joined_all_df["window_contrib_amount"] / joined_all_df["window_days"]) /
    joined_all_df["avg_daily_contrib_amount"]
)
# Pair 2: normalizing by this month's average daily performance
joined_all_df["month_daily_count_score"] = (
    (joined_all_df["window_contrib_count"] / joined_all_df["window_days"]) /
    joined_all_df["avg_month_daily_contrib_count"]
)
joined_all_df["month_daily_amount_score"] = (
    (joined_all_df["window_contrib_amount"] / joined_all_df["window_days"]) /
    joined_all_df["avg_month_daily_contrib_amount"]
)
# Pair 3: normalizing by average daily performance to date (from beginning of the period)
joined_all_df["avg_daily_td_count_score"] = (
    (joined_all_df["window_contrib_count"] / joined_all_df["window_days"]) /
    joined_all_df["avg_daily_contrib_count_td"]
)
joined_all_df["avg_daily_td_amount_score"] = (
    (joined_all_df["window_contrib_amount"] / joined_all_df["window_days"]) /
    joined_all_df["avg_daily_contrib_amount_td"]
)

# Fill missing values for quantitative columns & format them
num_cols_int = [
    "window_days", "contrib_count", "avg_daily_contrib_count",
    "window_contrib_count", "avg_month_daily_contrib_count"
]
num_cols_float = [
    "contrib_amount", "avg_daily_contrib_amount", "window_contrib_amount",
    "avg_daily_fp_count_score", "avg_daily_fp_amount_score",
    "month_daily_count_score", "month_daily_amount_score",
    "avg_daily_td_count_score", "avg_daily_td_amount_score",
    "avg_month_daily_contrib_amount",
    "avg_daily_contrib_count_td", "avg_daily_contrib_amount_td"
]

# Fill NAs
for col in num_cols_int:
    if col in joined_all_df.columns:
        joined_all_df[col] = joined_all_df[col].fillna(0).astype(int)
for col in num_cols_float:
    if col in joined_all_df.columns:
        joined_all_df[col] = joined_all_df[col].fillna(0).astype(float).map("{:.2f}".format)

# print(dc_inbox_sum_df.columns)
joined_all_df.head()
# print(window_col_df.columns)

window_col_df:  122458
dc_inbox_sum_df:  25785
joined_all_df:  25785


,dc_name,dc_date,newsletter_count,body_word_count_sum,fundraising_terms_sum,fundraising_ratio_mean,total_newsletters,dc_congress,dc_gender,dc_state,...,avg_daily_contrib_amount_td,window_days,window_contrib_count,window_contrib_amount,avg_daily_fp_count_score,avg_daily_fp_amount_score,month_daily_count_score,month_daily_amount_score,avg_daily_td_count_score,avg_daily_td_amount_score
0,A. FERGUSON,2019-01-13,1,357,0,0.000000,44,116,M,GA,...,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,A. FERGUSON,2019-01-27,1,528,0,0.000000,44,116,M,GA,...,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,A. FERGUSON,2019-02-10,1,457,0,0.000000,44,116,M,GA,...,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,A. FERGUSON,2019-03-03,1,430,1,0.002326,44,116,M,GA,...,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,A. FERGUSON,2019-03-17,1,271,1,0.003690,44,116,M,GA,...,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [114]:
# Output the final joined file to a CSV file

joined_all_df.to_csv(
    "../data/matched/joined_all_2020_test.csv",
    index=False,  # no row numbers
    encoding="latin1"
)